## Making Trained Model table more flexible

In [1]:
import datajoint as dj
dj.config['schema_name'] = "mohammad_ttmdemo"
from nnfabrik.main import *
from nnfabrik.template import TransferredTrainedModelBase

Connecting mohammad@sinzlab.chlkmukhxp6i.eu-central-1.rds.amazonaws.com:3306


In [2]:
@schema
class TransferredTrainedModel(TransferredTrainedModelBase):
    table_comment = "Transferred trained models"

## Define recipe table(s)

In [3]:
@schema
class ModelTransferRecipe(dj.Lookup):
    definition = """
    transfer_step: int
    -> Model
    -> Model.proj(prev_model_fn='model_fn', prev_model_hash='model_hash')
    """
    
    @property
    def post_restr(self):
        """
        This restriction clause is used to make sure that aside from switching trainers, the utilized model is to remain
        the same. 
        """
        return 'trainer_fn = prev_trainer_fn'

    def add_entry(self, recipe):
        entry = dict(**{f"prev_{k}": v for k, v in recipe['transfer_from'].items()}, **recipe['transfer_to'])
        entry['transfer_step'] = recipe['transfer_step']
            
        self.insert1(entry)

In [4]:
@schema
class TrainerTransferRecipe(dj.Lookup):
    definition = """
    transfer_step: int
    -> Trainer
    -> Trainer.proj(prev_trainer_fn='trainer_fn', prev_trainer_hash='trainer_hash')
    """
    
    @property
    def post_restr(self):
        """
        This restriction clause is used to make sure that aside from switching trainers, the utilized model is to remain
        the same. 
        """
        return 'model_fn = prev_model_fn'

    def add_entry(self, recipe):
        entry = dict(**{f"prev_{k}": v for k, v in recipe['transfer_from'].items()}, **recipe['transfer_to'])
        entry['transfer_step'] = recipe['transfer_step']
            
        self.insert1(entry)

## Add some entries to the table (usual step)

In [5]:
Fabrikant().insert1(dict(fabrikant_name='Mohammad Bashiri',
                         email="mohammadbashiri93@gmail.com",
                         affiliation='sinzlab',
                         dj_username="mohammad"))

In [6]:
Fabrikant()

fabrikant_name Name of the contributor that added this entry,email e-mail address,affiliation conributor's affiliation,dj_username DataJoint username
Mohammad Bashiri,mohammadbashiri93@gmail.com,sinzlab,mohammad


In [7]:
dataset_fn = "learn_tuning.datasets.real.mouse_static_loaders"
paths = ["/notebooks/data/Static/static20457-5-9-preproc0.h5"]
dataset_config = dict(paths=paths, batch_size=64, neurons_n=2, device='cuda')

Dataset().add_entry(dataset_fn=dataset_fn, dataset_config=dataset_config, dataset_fabrikant="Mohammad Bashiri", dataset_comment="beautiful dataset");

In [8]:
Dataset()

dataset_fn name of the dataset loader function,dataset_hash hash of the configuration object,dataset_config dataset configuration object,dataset_fabrikant Name of the contributor that added this entry,dataset_comment short description,dataset_ts UTZ timestamp at time of insertion
learn_tuning.datasets.real.mouse_static_loaders,63a570842dc1efa062d72c507a3f7348,=BLOB=,Mohammad Bashiri,beautiful dataset,2020-04-06 15:12:34


In [9]:
model_fn = "learn_tuning.models.neural.affine_encoder"
model_config = dict(filters_n=8, device='cuda')

Model().add_entry(model_fn=model_fn, model_config=model_config, model_fabrikant="Mohammad Bashiri", model_comment="SOA Model");

In [10]:
Model()

model_fn name of the model function,model_hash hash of the model configuration,model_config model configuration to be passed into the function,model_fabrikant Name of the contributor that added this entry,model_comment short description,model_ts UTZ timestamp at time of insertion
learn_tuning.models.neural.affine_encoder,dc8b9e3a7843a4d30186a58af56eb81c,=BLOB=,Mohammad Bashiri,SOA Model,2020-04-06 15:12:34


In [11]:
trainer_fn = "learn_tuning.trainers.affine_trainer"
trainer_config = dict(lr=0.005, epochs=2, evice='cuda')

Trainer().add_entry(trainer_fn=trainer_fn, trainer_config=trainer_config, trainer_fabrikant="Mohammad Bashiri", trainer_comment="professional trainer");

In [12]:
Trainer()

trainer_fn name of the Trainer loader function,trainer_hash hash of the configuration object,trainer_config training configuration object,trainer_fabrikant Name of the contributor that added this entry,trainer_comment short description,trainer_ts UTZ timestamp at time of insertion
learn_tuning.trainers.affine_trainer,281e54bbfa39beda082739f5bea78d84,=BLOB=,Mohammad Bashiri,professional trainer,2020-04-06 15:12:34


In [13]:
Seed().insert([{'seed':7}])

In [14]:
Seed()

seed Random seed that is passed to the model- and dataset-builder
7


In [15]:
TransferredTrainedModel()

transfer_step transfer step,model_fn name of the model function,model_hash hash of the model configuration,dataset_fn name of the dataset loader function,dataset_hash hash of the configuration object,trainer_fn name of the Trainer loader function,trainer_hash hash of the configuration object,seed Random seed that is passed to the model- and dataset-builder,prev_model_fn,prev_model_hash,prev_dataset_fn,prev_dataset_hash,prev_trainer_fn,prev_trainer_hash,comment short description,score loss,output trainer object's output,fabrikant_name Name of the contributor that added this entry,trainedmodel_ts UTZ timestamp at time of insertion,current_model_fn,current_model_hash,current_dataset_fn,current_dataset_hash,current_trainer_fn,current_trainer_hash


Below are the possible cases where one might want to transfer something from an existing set of trained models to a new set of trained models:
* you trained models with a specific trainer and you want to continue training them with a new trainer
* you trained models with a specific dataset and you want to 

This boils down to:
* what datasets and/or models and/or trainers you would like to use at a specific transfer step -> specify this in the recipe
* which already-exisiting entries you would like this transfer step to be applied on -> specify this in the `populate` restrictions

## Populating TrainedModel normally

In [16]:
TransferredTrainedModel.populate()

100%|██████████| 1/1 [00:00<00:00, 131.91it/s]

Score improved (nan --> 0.029949).



100%|██████████| 70/70 [00:00<00:00, 93.21it/s]


 Epoch 1/2 | train loss: 19.442425 | val loss: 3.319493 | train corr: 0.191348 | val corr: 0.091619 | test corr: 0.119776
Score improved (0.029949 --> 0.091619).


100%|██████████| 70/70 [00:00<00:00, 99.04it/s]


 Epoch 2/2 | train loss: 4.674604 | val loss: 3.308230 | train corr: 0.277520 | val corr: 0.160593 | test corr: 0.158540
Score improved (0.091619 --> 0.160593).
Retrieve best model..


In [17]:
TransferredTrainedModel()

transfer_step transfer step,model_fn name of the model function,model_hash hash of the model configuration,dataset_fn name of the dataset loader function,dataset_hash hash of the configuration object,trainer_fn name of the Trainer loader function,trainer_hash hash of the configuration object,seed Random seed that is passed to the model- and dataset-builder,prev_model_fn,prev_model_hash,prev_dataset_fn,prev_dataset_hash,prev_trainer_fn,prev_trainer_hash,comment short description,score loss,output trainer object's output,fabrikant_name Name of the contributor that added this entry,trainedmodel_ts UTZ timestamp at time of insertion,current_model_fn,current_model_hash,current_dataset_fn,current_dataset_hash,current_trainer_fn,current_trainer_hash
0,learn_tuning.models.neural.affine_encoder,dc8b9e3a7843a4d30186a58af56eb81c,learn_tuning.datasets.real.mouse_static_loaders,63a570842dc1efa062d72c507a3f7348,learn_tuning.trainers.affine_trainer,281e54bbfa39beda082739f5bea78d84,7,,,,,,,professional trainer.SOA Model.beautiful dataset,0.15854,=BLOB=,Mohammad Bashiri,2020-04-06 15:12:50,learn_tuning.models.neural.affine_encoder,dc8b9e3a7843a4d30186a58af56eb81c,learn_tuning.datasets.real.mouse_static_loaders,63a570842dc1efa062d72c507a3f7348,learn_tuning.trainers.affine_trainer,281e54bbfa39beda082739f5bea78d84


## Adding recipes after populating TrainedModel

First, let's add the model that we want to use for the transfer. 

In [18]:
model_fn = "learn_tuning.models.neural.affine_encoder"
model_config = dict(filters_n=16, device='cuda')

Model().add_entry(model_fn=model_fn, model_config=model_config, model_fabrikant="Mohammad Bashiri", model_comment="SOA Model");

In [19]:
Model()

model_fn name of the model function,model_hash hash of the model configuration,model_config model configuration to be passed into the function,model_fabrikant Name of the contributor that added this entry,model_comment short description,model_ts UTZ timestamp at time of insertion
learn_tuning.models.neural.affine_encoder,6dab24297b5f3d3adb5d44a6a2a18cf9,=BLOB=,Mohammad Bashiri,SOA Model,2020-04-06 15:12:51
learn_tuning.models.neural.affine_encoder,dc8b9e3a7843a4d30186a58af56eb81c,=BLOB=,Mohammad Bashiri,SOA Model,2020-04-06 15:12:34


#### Define the recipe

In [20]:
transfer_from = {"model_fn": 'learn_tuning.models.neural.affine_encoder', "model_hash": 'dc8b9e3a7843a4d30186a58af56eb81c'}
transfer_to = {"model_fn": 'learn_tuning.models.neural.affine_encoder', "model_hash": '6dab24297b5f3d3adb5d44a6a2a18cf9'}
transfer_recipe = {"transfer_step": 1, "transfer_from": transfer_from, "transfer_to": transfer_to}

In [21]:
ModelTransferRecipe().add_entry(transfer_recipe)

In [22]:
ModelTransferRecipe()

transfer_step,model_fn name of the model function,model_hash hash of the model configuration,prev_model_fn name of the model function,prev_model_hash hash of the model configuration
1,learn_tuning.models.neural.affine_encoder,6dab24297b5f3d3adb5d44a6a2a18cf9,learn_tuning.models.neural.affine_encoder,dc8b9e3a7843a4d30186a58af56eb81c


Let's also add a trainer for the next transfer step (e.g. step 2). 

In [23]:
trainer_fn = "learn_tuning.trainers.affine_trainer"
trainer_config = dict(lr=0.05, epochs=2, evice='cuda')

Trainer().add_entry(trainer_fn=trainer_fn, trainer_config=trainer_config, trainer_fabrikant="Mohammad Bashiri", trainer_comment="professional trainer");

In [24]:
Trainer()

trainer_fn name of the Trainer loader function,trainer_hash hash of the configuration object,trainer_config training configuration object,trainer_fabrikant Name of the contributor that added this entry,trainer_comment short description,trainer_ts UTZ timestamp at time of insertion
learn_tuning.trainers.affine_trainer,1879c35492a8788d5b6eea7d4a485fdd,=BLOB=,Mohammad Bashiri,professional trainer,2020-04-06 15:12:51
learn_tuning.trainers.affine_trainer,281e54bbfa39beda082739f5bea78d84,=BLOB=,Mohammad Bashiri,professional trainer,2020-04-06 15:12:34


In [25]:
transfer_from = {"trainer_fn": 'learn_tuning.trainers.affine_trainer', "trainer_hash": '281e54bbfa39beda082739f5bea78d84'}
transfer_to = {"trainer_fn": 'learn_tuning.trainers.affine_trainer', "trainer_hash": '1879c35492a8788d5b6eea7d4a485fdd'}
transfer_recipe = {"transfer_step": 2, "transfer_from": transfer_from, "transfer_to": transfer_to}

In [26]:
TrainerTransferRecipe().add_entry(transfer_recipe)

In [27]:
TrainerTransferRecipe()

transfer_step,trainer_fn name of the Trainer loader function,trainer_hash hash of the configuration object,prev_trainer_fn name of the Trainer loader function,prev_trainer_hash hash of the configuration object
2,learn_tuning.trainers.affine_trainer,1879c35492a8788d5b6eea7d4a485fdd,learn_tuning.trainers.affine_trainer,281e54bbfa39beda082739f5bea78d84


### Add the recipes to the table

In [28]:
TransferredTrainedModel.transfer_recipe = [TrainerTransferRecipe(), ModelTransferRecipe()]

### Populate the table for transfer step 1

In [29]:
TransferredTrainedModel.populate()

100%|██████████| 1/1 [00:00<00:00, 375.87it/s]

Score improved (nan --> -0.029592).



100%|██████████| 70/70 [00:00<00:00, 96.44it/s]


 Epoch 1/2 | train loss: 17.693577 | val loss: 3.497843 | train corr: -0.003003 | val corr: -0.007400 | test corr: -0.008406
Score improved (-0.029592 --> -0.007400).


100%|██████████| 70/70 [00:00<00:00, 97.27it/s]


...

Score improved (nan --> -0.029592).



100%|██████████| 70/70 [00:00<00:00, 97.52it/s]


 Epoch 1/2 | train loss: 17.693577 | val loss: 3.228006 | train corr: 0.053832 | val corr: 0.019069 | test corr: 0.010965
Score improved (-0.029592 --> 0.019069).


100%|██████████| 70/70 [00:00<00:00, 96.31it/s]


 Epoch 2/2 | train loss: 6.074929 | val loss: 3.450161 | train corr: 0.129775 | val corr: 0.070964 | test corr: 0.021818
Score improved (0.019069 --> 0.070964).
Retrieve best model..


In [30]:
TransferredTrainedModel()

transfer_step transfer step,model_fn name of the model function,model_hash hash of the model configuration,dataset_fn name of the dataset loader function,dataset_hash hash of the configuration object,trainer_fn name of the Trainer loader function,trainer_hash hash of the configuration object,seed Random seed that is passed to the model- and dataset-builder,prev_model_fn,prev_model_hash,prev_dataset_fn,prev_dataset_hash,prev_trainer_fn,prev_trainer_hash,comment short description,score loss,output trainer object's output,fabrikant_name Name of the contributor that added this entry,trainedmodel_ts UTZ timestamp at time of insertion,current_model_fn,current_model_hash,current_dataset_fn,current_dataset_hash,current_trainer_fn,current_trainer_hash
0,learn_tuning.models.neural.affine_encoder,dc8b9e3a7843a4d30186a58af56eb81c,learn_tuning.datasets.real.mouse_static_loaders,63a570842dc1efa062d72c507a3f7348,learn_tuning.trainers.affine_trainer,281e54bbfa39beda082739f5bea78d84,7,,,,,,,professional trainer.SOA Model.beautiful dataset,0.15854,=BLOB=,Mohammad Bashiri,2020-04-06 15:12:50,learn_tuning.models.neural.affine_encoder,dc8b9e3a7843a4d30186a58af56eb81c,learn_tuning.datasets.real.mouse_static_loaders,63a570842dc1efa062d72c507a3f7348,learn_tuning.trainers.affine_trainer,281e54bbfa39beda082739f5bea78d84
1,learn_tuning.models.neural.affine_encoder,6dab24297b5f3d3adb5d44a6a2a18cf9,learn_tuning.datasets.real.mouse_static_loaders,63a570842dc1efa062d72c507a3f7348,learn_tuning.trainers.affine_trainer,1879c35492a8788d5b6eea7d4a485fdd,7,learn_tuning.models.neural.affine_encoder,dc8b9e3a7843a4d30186a58af56eb81c,learn_tuning.datasets.real.mouse_static_loaders,63a570842dc1efa062d72c507a3f7348,learn_tuning.trainers.affine_trainer,281e54bbfa39beda082739f5bea78d84,professional trainer.SOA Model.beautiful dataset,-0.00758789,=BLOB=,Mohammad Bashiri,2020-04-06 15:13:03,learn_tuning.models.neural.affine_encoder,6dab24297b5f3d3adb5d44a6a2a18cf9,learn_tuning.datasets.real.mouse_static_loaders,63a570842dc1efa062d72c507a3f7348,learn_tuning.trainers.affine_trainer,1879c35492a8788d5b6eea7d4a485fdd
1,learn_tuning.models.neural.affine_encoder,6dab24297b5f3d3adb5d44a6a2a18cf9,learn_tuning.datasets.real.mouse_static_loaders,63a570842dc1efa062d72c507a3f7348,learn_tuning.trainers.affine_trainer,281e54bbfa39beda082739f5bea78d84,7,learn_tuning.models.neural.affine_encoder,dc8b9e3a7843a4d30186a58af56eb81c,learn_tuning.datasets.real.mouse_static_loaders,63a570842dc1efa062d72c507a3f7348,learn_tuning.trainers.affine_trainer,281e54bbfa39beda082739f5bea78d84,professional trainer.SOA Model.beautiful dataset,0.0218181,=BLOB=,Mohammad Bashiri,2020-04-06 15:13:16,learn_tuning.models.neural.affine_encoder,6dab24297b5f3d3adb5d44a6a2a18cf9,learn_tuning.datasets.real.mouse_static_loaders,63a570842dc1efa062d72c507a3f7348,learn_tuning.trainers.affine_trainer,281e54bbfa39beda082739f5bea78d84


### Populate the table for transfer step 2

In [31]:
TransferredTrainedModel.populate()

100%|██████████| 1/1 [00:00<00:00, 430.36it/s]

Score improved (nan --> -0.029592).



100%|██████████| 70/70 [00:00<00:00, 94.02it/s]


 Epoch 1/2 | train loss: 17.693577 | val loss: 3.497846 | train corr: -0.003082 | val corr: -0.007384 | test corr: -0.008357
Score improved (-0.029592 --> -0.007384).


100%|██████████| 70/70 [00:00<00:00, 96.48it/s]


...

Score improved (nan --> 0.029949).



100%|██████████| 70/70 [00:00<00:00, 94.39it/s]


 Epoch 1/2 | train loss: 19.442425 | val loss: 3.288859 | train corr: 0.011899 | val corr: 0.002145 | test corr: -0.034141
EarlyStopping counter: 1 out of 20


100%|██████████| 70/70 [00:00<00:00, 95.18it/s]


 Epoch 2/2 | train loss: 5.852595 | val loss: 3.165433 | train corr: 0.063888 | val corr: 0.083975 | test corr: 0.010772
Score improved (0.029949 --> 0.083975).
Retrieve best model..


In [32]:
TransferredTrainedModel()

transfer_step transfer step,model_fn name of the model function,model_hash hash of the model configuration,dataset_fn name of the dataset loader function,dataset_hash hash of the configuration object,trainer_fn name of the Trainer loader function,trainer_hash hash of the configuration object,seed Random seed that is passed to the model- and dataset-builder,prev_model_fn,prev_model_hash,prev_dataset_fn,prev_dataset_hash,prev_trainer_fn,prev_trainer_hash,comment short description,score loss,output trainer object's output,fabrikant_name Name of the contributor that added this entry,trainedmodel_ts UTZ timestamp at time of insertion,current_model_fn,current_model_hash,current_dataset_fn,current_dataset_hash,current_trainer_fn,current_trainer_hash
0,learn_tuning.models.neural.affine_encoder,dc8b9e3a7843a4d30186a58af56eb81c,learn_tuning.datasets.real.mouse_static_loaders,63a570842dc1efa062d72c507a3f7348,learn_tuning.trainers.affine_trainer,281e54bbfa39beda082739f5bea78d84,7,,,,,,,professional trainer.SOA Model.beautiful dataset,0.15854,=BLOB=,Mohammad Bashiri,2020-04-06 15:12:50,learn_tuning.models.neural.affine_encoder,dc8b9e3a7843a4d30186a58af56eb81c,learn_tuning.datasets.real.mouse_static_loaders,63a570842dc1efa062d72c507a3f7348,learn_tuning.trainers.affine_trainer,281e54bbfa39beda082739f5bea78d84
1,learn_tuning.models.neural.affine_encoder,6dab24297b5f3d3adb5d44a6a2a18cf9,learn_tuning.datasets.real.mouse_static_loaders,63a570842dc1efa062d72c507a3f7348,learn_tuning.trainers.affine_trainer,1879c35492a8788d5b6eea7d4a485fdd,7,learn_tuning.models.neural.affine_encoder,dc8b9e3a7843a4d30186a58af56eb81c,learn_tuning.datasets.real.mouse_static_loaders,63a570842dc1efa062d72c507a3f7348,learn_tuning.trainers.affine_trainer,281e54bbfa39beda082739f5bea78d84,professional trainer.SOA Model.beautiful dataset,-0.00758789,=BLOB=,Mohammad Bashiri,2020-04-06 15:13:03,learn_tuning.models.neural.affine_encoder,6dab24297b5f3d3adb5d44a6a2a18cf9,learn_tuning.datasets.real.mouse_static_loaders,63a570842dc1efa062d72c507a3f7348,learn_tuning.trainers.affine_trainer,1879c35492a8788d5b6eea7d4a485fdd
1,learn_tuning.models.neural.affine_encoder,6dab24297b5f3d3adb5d44a6a2a18cf9,learn_tuning.datasets.real.mouse_static_loaders,63a570842dc1efa062d72c507a3f7348,learn_tuning.trainers.affine_trainer,281e54bbfa39beda082739f5bea78d84,7,learn_tuning.models.neural.affine_encoder,dc8b9e3a7843a4d30186a58af56eb81c,learn_tuning.datasets.real.mouse_static_loaders,63a570842dc1efa062d72c507a3f7348,learn_tuning.trainers.affine_trainer,281e54bbfa39beda082739f5bea78d84,professional trainer.SOA Model.beautiful dataset,0.0218181,=BLOB=,Mohammad Bashiri,2020-04-06 15:13:16,learn_tuning.models.neural.affine_encoder,6dab24297b5f3d3adb5d44a6a2a18cf9,learn_tuning.datasets.real.mouse_static_loaders,63a570842dc1efa062d72c507a3f7348,learn_tuning.trainers.affine_trainer,281e54bbfa39beda082739f5bea78d84
2,learn_tuning.models.neural.affine_encoder,6dab24297b5f3d3adb5d44a6a2a18cf9,learn_tuning.datasets.real.mouse_static_loaders,63a570842dc1efa062d72c507a3f7348,learn_tuning.trainers.affine_trainer,1879c35492a8788d5b6eea7d4a485fdd,7,learn_tuning.models.neural.affine_encoder,6dab24297b5f3d3adb5d44a6a2a18cf9,learn_tuning.datasets.real.mouse_static_loaders,63a570842dc1efa062d72c507a3f7348,learn_tuning.trainers.affine_trainer,281e54bbfa39beda082739f5bea78d84,professional trainer.SOA Model.beautiful dataset,-0.00751,=BLOB=,Mohammad Bashiri,2020-04-06 15:13:28,learn_tuning.models.neural.affine_encoder,6dab24297b5f3d3adb5d44a6a2a18cf9,learn_tuning.datasets.real.mouse_static_loaders,63a570842dc1efa062d72c507a3f7348,learn_tuning.trainers.affine_trainer,1879c35492a8788d5b6eea7d4a485fdd
2,learn_tuning.models.neural.affine_encoder,dc8b9e3a7843a4d30186a58af56eb81c,learn_tuning.datasets.real.mouse_static_loaders,63a570842dc1efa062d72c507a3f7348,learn_tuning.trainers.affine_trainer,1879c35492a8788d5b6eea7d4a485fdd,7,learn_tuning.models.neural.affine_encoder,6dab24297b5f3d3adb5d44a6a2a18cf9,learn_tuning.datasets.r

## Creating recipes and populating TrainedModel table from skratch

Now let's do the above, but spcify the recipe in the very beginning (specify the recipes before populating the table at all). So we have:
* all our tables 
* recipes (same as above), and
* added the recipes to the table

First, let's remove the entries from the `TrainedModel` table

In [33]:
TransferredTrainedModel.delete()

About to delete:
`mohammad_example`.`__transferred_trained_model__model_storage`: 5 items
`mohammad_example`.`__transferred_trained_model`: 5 items


Proceed? [yes, No]:  yes


Committed.


In [34]:
Dataset()

dataset_fn name of the dataset loader function,dataset_hash hash of the configuration object,dataset_config dataset configuration object,dataset_fabrikant Name of the contributor that added this entry,dataset_comment short description,dataset_ts UTZ timestamp at time of insertion
learn_tuning.datasets.real.mouse_static_loaders,63a570842dc1efa062d72c507a3f7348,=BLOB=,Mohammad Bashiri,beautiful dataset,2020-04-06 15:12:34


In [35]:
Model()

model_fn name of the model function,model_hash hash of the model configuration,model_config model configuration to be passed into the function,model_fabrikant Name of the contributor that added this entry,model_comment short description,model_ts UTZ timestamp at time of insertion
learn_tuning.models.neural.affine_encoder,6dab24297b5f3d3adb5d44a6a2a18cf9,=BLOB=,Mohammad Bashiri,SOA Model,2020-04-06 15:12:51
learn_tuning.models.neural.affine_encoder,dc8b9e3a7843a4d30186a58af56eb81c,=BLOB=,Mohammad Bashiri,SOA Model,2020-04-06 15:12:34


In [36]:
Trainer()

trainer_fn name of the Trainer loader function,trainer_hash hash of the configuration object,trainer_config training configuration object,trainer_fabrikant Name of the contributor that added this entry,trainer_comment short description,trainer_ts UTZ timestamp at time of insertion
learn_tuning.trainers.affine_trainer,1879c35492a8788d5b6eea7d4a485fdd,=BLOB=,Mohammad Bashiri,professional trainer,2020-04-06 15:12:51
learn_tuning.trainers.affine_trainer,281e54bbfa39beda082739f5bea78d84,=BLOB=,Mohammad Bashiri,professional trainer,2020-04-06 15:12:34


In [37]:
ModelTransferRecipe()

transfer_step,model_fn name of the model function,model_hash hash of the model configuration,prev_model_fn name of the model function,prev_model_hash hash of the model configuration
1,learn_tuning.models.neural.affine_encoder,6dab24297b5f3d3adb5d44a6a2a18cf9,learn_tuning.models.neural.affine_encoder,dc8b9e3a7843a4d30186a58af56eb81c


In [38]:
TrainerTransferRecipe()

transfer_step,trainer_fn name of the Trainer loader function,trainer_hash hash of the configuration object,prev_trainer_fn name of the Trainer loader function,prev_trainer_hash hash of the configuration object
2,learn_tuning.trainers.affine_trainer,1879c35492a8788d5b6eea7d4a485fdd,learn_tuning.trainers.affine_trainer,281e54bbfa39beda082739f5bea78d84


And the `TrainedModel` table is empty:

In [39]:
TransferredTrainedModel()

transfer_step transfer step,model_fn name of the model function,model_hash hash of the model configuration,dataset_fn name of the dataset loader function,dataset_hash hash of the configuration object,trainer_fn name of the Trainer loader function,trainer_hash hash of the configuration object,seed Random seed that is passed to the model- and dataset-builder,prev_model_fn,prev_model_hash,prev_dataset_fn,prev_dataset_hash,prev_trainer_fn,prev_trainer_hash,comment short description,score loss,output trainer object's output,fabrikant_name Name of the contributor that added this entry,trainedmodel_ts UTZ timestamp at time of insertion,current_model_fn,current_model_hash,current_dataset_fn,current_dataset_hash,current_trainer_fn,current_trainer_hash


Apply necessary restrictions for transfer step 0:

In [40]:
TransferredTrainedModel.populate("model_hash = 'dc8b9e3a7843a4d30186a58af56eb81c'", "trainer_hash = '281e54bbfa39beda082739f5bea78d84'")

100%|██████████| 1/1 [00:00<00:00, 463.05it/s]

Score improved (nan --> 0.029949).



100%|██████████| 70/70 [00:00<00:00, 96.34it/s]


 Epoch 1/2 | train loss: 19.442425 | val loss: 3.319487 | train corr: 0.191337 | val corr: 0.091535 | test corr: 0.119750
Score improved (0.029949 --> 0.091535).


100%|██████████| 70/70 [00:00<00:00, 98.13it/s]


 Epoch 2/2 | train loss: 4.675240 | val loss: 3.309483 | train corr: 0.278635 | val corr: 0.161760 | test corr: 0.159281
Score improved (0.091535 --> 0.161760).
Retrieve best model..


In [41]:
TransferredTrainedModel()

transfer_step transfer step,model_fn name of the model function,model_hash hash of the model configuration,dataset_fn name of the dataset loader function,dataset_hash hash of the configuration object,trainer_fn name of the Trainer loader function,trainer_hash hash of the configuration object,seed Random seed that is passed to the model- and dataset-builder,prev_model_fn,prev_model_hash,prev_dataset_fn,prev_dataset_hash,prev_trainer_fn,prev_trainer_hash,comment short description,score loss,output trainer object's output,fabrikant_name Name of the contributor that added this entry,trainedmodel_ts UTZ timestamp at time of insertion,current_model_fn,current_model_hash,current_dataset_fn,current_dataset_hash,current_trainer_fn,current_trainer_hash
0,learn_tuning.models.neural.affine_encoder,dc8b9e3a7843a4d30186a58af56eb81c,learn_tuning.datasets.real.mouse_static_loaders,63a570842dc1efa062d72c507a3f7348,learn_tuning.trainers.affine_trainer,281e54bbfa39beda082739f5bea78d84,7,,,,,,,professional trainer.SOA Model.beautiful dataset,0.159281,=BLOB=,Mohammad Bashiri,2020-04-06 15:13:56,learn_tuning.models.neural.affine_encoder,dc8b9e3a7843a4d30186a58af56eb81c,learn_tuning.datasets.real.mouse_static_loaders,63a570842dc1efa062d72c507a3f7348,learn_tuning.trainers.affine_trainer,281e54bbfa39beda082739f5bea78d84


In [42]:
TransferredTrainedModel.populate()

100%|██████████| 1/1 [00:00<00:00, 427.82it/s]

Score improved (nan --> -0.029592).



100%|██████████| 70/70 [00:00<00:00, 95.54it/s]


 Epoch 1/2 | train loss: 17.693577 | val loss: 3.497870 | train corr: -0.003179 | val corr: -0.006904 | test corr: -0.015637
Score improved (-0.029592 --> -0.006904).


100%|██████████| 70/70 [00:00<00:00, 93.99it/s]


...

Score improved (nan --> -0.029592).



100%|██████████| 70/70 [00:00<00:00, 94.39it/s]


 Epoch 1/2 | train loss: 17.693577 | val loss: 3.225005 | train corr: 0.055381 | val corr: 0.019143 | test corr: 0.013396
Score improved (-0.029592 --> 0.019143).


100%|██████████| 70/70 [00:00<00:00, 96.11it/s]


 Epoch 2/2 | train loss: 6.073028 | val loss: 3.348546 | train corr: 0.122133 | val corr: 0.061679 | test corr: 0.010412
Score improved (0.019143 --> 0.061679).
Retrieve best model..


In [43]:
TransferredTrainedModel()

transfer_step transfer step,model_fn name of the model function,model_hash hash of the model configuration,dataset_fn name of the dataset loader function,dataset_hash hash of the configuration object,trainer_fn name of the Trainer loader function,trainer_hash hash of the configuration object,seed Random seed that is passed to the model- and dataset-builder,prev_model_fn,prev_model_hash,prev_dataset_fn,prev_dataset_hash,prev_trainer_fn,prev_trainer_hash,comment short description,score loss,output trainer object's output,fabrikant_name Name of the contributor that added this entry,trainedmodel_ts UTZ timestamp at time of insertion,current_model_fn,current_model_hash,current_dataset_fn,current_dataset_hash,current_trainer_fn,current_trainer_hash
0,learn_tuning.models.neural.affine_encoder,dc8b9e3a7843a4d30186a58af56eb81c,learn_tuning.datasets.real.mouse_static_loaders,63a570842dc1efa062d72c507a3f7348,learn_tuning.trainers.affine_trainer,281e54bbfa39beda082739f5bea78d84,7,,,,,,,professional trainer.SOA Model.beautiful dataset,0.159281,=BLOB=,Mohammad Bashiri,2020-04-06 15:13:56,learn_tuning.models.neural.affine_encoder,dc8b9e3a7843a4d30186a58af56eb81c,learn_tuning.datasets.real.mouse_static_loaders,63a570842dc1efa062d72c507a3f7348,learn_tuning.trainers.affine_trainer,281e54bbfa39beda082739f5bea78d84
1,learn_tuning.models.neural.affine_encoder,6dab24297b5f3d3adb5d44a6a2a18cf9,learn_tuning.datasets.real.mouse_static_loaders,63a570842dc1efa062d72c507a3f7348,learn_tuning.trainers.affine_trainer,1879c35492a8788d5b6eea7d4a485fdd,7,learn_tuning.models.neural.affine_encoder,dc8b9e3a7843a4d30186a58af56eb81c,learn_tuning.datasets.real.mouse_static_loaders,63a570842dc1efa062d72c507a3f7348,learn_tuning.trainers.affine_trainer,281e54bbfa39beda082739f5bea78d84,professional trainer.SOA Model.beautiful dataset,-0.00833925,=BLOB=,Mohammad Bashiri,2020-04-06 15:14:09,learn_tuning.models.neural.affine_encoder,6dab24297b5f3d3adb5d44a6a2a18cf9,learn_tuning.datasets.real.mouse_static_loaders,63a570842dc1efa062d72c507a3f7348,learn_tuning.trainers.affine_trainer,1879c35492a8788d5b6eea7d4a485fdd
1,learn_tuning.models.neural.affine_encoder,6dab24297b5f3d3adb5d44a6a2a18cf9,learn_tuning.datasets.real.mouse_static_loaders,63a570842dc1efa062d72c507a3f7348,learn_tuning.trainers.affine_trainer,281e54bbfa39beda082739f5bea78d84,7,learn_tuning.models.neural.affine_encoder,dc8b9e3a7843a4d30186a58af56eb81c,learn_tuning.datasets.real.mouse_static_loaders,63a570842dc1efa062d72c507a3f7348,learn_tuning.trainers.affine_trainer,281e54bbfa39beda082739f5bea78d84,professional trainer.SOA Model.beautiful dataset,0.0133964,=BLOB=,Mohammad Bashiri,2020-04-06 15:14:21,learn_tuning.models.neural.affine_encoder,6dab24297b5f3d3adb5d44a6a2a18cf9,learn_tuning.datasets.real.mouse_static_loaders,63a570842dc1efa062d72c507a3f7348,learn_tuning.trainers.affine_trainer,281e54bbfa39beda082739f5bea78d84


In [44]:
TransferredTrainedModel.populate()

100%|██████████| 1/1 [00:00<00:00, 411.81it/s]

Score improved (nan --> -0.029592).



100%|██████████| 70/70 [00:00<00:00, 93.83it/s]


 Epoch 1/2 | train loss: 17.693577 | val loss: 3.497863 | train corr: -0.003137 | val corr: -0.007177 | test corr: -0.008629
Score improved (-0.029592 --> -0.007177).


100%|██████████| 70/70 [00:00<00:00, 93.81it/s]


...

Score improved (nan --> 0.029949).



100%|██████████| 70/70 [00:00<00:00, 93.40it/s]


 Epoch 1/2 | train loss: 19.442425 | val loss: 3.240143 | train corr: 0.049282 | val corr: 0.039142 | test corr: -0.001447
Score improved (0.029949 --> 0.039142).


100%|██████████| 70/70 [00:00<00:00, 95.06it/s]


 Epoch 2/2 | train loss: 5.551387 | val loss: 3.236351 | train corr: 0.005880 | val corr: 0.034914 | test corr: -0.021206
EarlyStopping counter: 1 out of 20
Retrieve best model..


In [45]:
TransferredTrainedModel()

transfer_step transfer step,model_fn name of the model function,model_hash hash of the model configuration,dataset_fn name of the dataset loader function,dataset_hash hash of the configuration object,trainer_fn name of the Trainer loader function,trainer_hash hash of the configuration object,seed Random seed that is passed to the model- and dataset-builder,prev_model_fn,prev_model_hash,prev_dataset_fn,prev_dataset_hash,prev_trainer_fn,prev_trainer_hash,comment short description,score loss,output trainer object's output,fabrikant_name Name of the contributor that added this entry,trainedmodel_ts UTZ timestamp at time of insertion,current_model_fn,current_model_hash,current_dataset_fn,current_dataset_hash,current_trainer_fn,current_trainer_hash
0,learn_tuning.models.neural.affine_encoder,dc8b9e3a7843a4d30186a58af56eb81c,learn_tuning.datasets.real.mouse_static_loaders,63a570842dc1efa062d72c507a3f7348,learn_tuning.trainers.affine_trainer,281e54bbfa39beda082739f5bea78d84,7,,,,,,,professional trainer.SOA Model.beautiful dataset,0.159281,=BLOB=,Mohammad Bashiri,2020-04-06 15:13:56,learn_tuning.models.neural.affine_encoder,dc8b9e3a7843a4d30186a58af56eb81c,learn_tuning.datasets.real.mouse_static_loaders,63a570842dc1efa062d72c507a3f7348,learn_tuning.trainers.affine_trainer,281e54bbfa39beda082739f5bea78d84
1,learn_tuning.models.neural.affine_encoder,6dab24297b5f3d3adb5d44a6a2a18cf9,learn_tuning.datasets.real.mouse_static_loaders,63a570842dc1efa062d72c507a3f7348,learn_tuning.trainers.affine_trainer,1879c35492a8788d5b6eea7d4a485fdd,7,learn_tuning.models.neural.affine_encoder,dc8b9e3a7843a4d30186a58af56eb81c,learn_tuning.datasets.real.mouse_static_loaders,63a570842dc1efa062d72c507a3f7348,learn_tuning.trainers.affine_trainer,281e54bbfa39beda082739f5bea78d84,professional trainer.SOA Model.beautiful dataset,-0.00833925,=BLOB=,Mohammad Bashiri,2020-04-06 15:14:09,learn_tuning.models.neural.affine_encoder,6dab24297b5f3d3adb5d44a6a2a18cf9,learn_tuning.datasets.real.mouse_static_loaders,63a570842dc1efa062d72c507a3f7348,learn_tuning.trainers.affine_trainer,1879c35492a8788d5b6eea7d4a485fdd
1,learn_tuning.models.neural.affine_encoder,6dab24297b5f3d3adb5d44a6a2a18cf9,learn_tuning.datasets.real.mouse_static_loaders,63a570842dc1efa062d72c507a3f7348,learn_tuning.trainers.affine_trainer,281e54bbfa39beda082739f5bea78d84,7,learn_tuning.models.neural.affine_encoder,dc8b9e3a7843a4d30186a58af56eb81c,learn_tuning.datasets.real.mouse_static_loaders,63a570842dc1efa062d72c507a3f7348,learn_tuning.trainers.affine_trainer,281e54bbfa39beda082739f5bea78d84,professional trainer.SOA Model.beautiful dataset,0.0133964,=BLOB=,Mohammad Bashiri,2020-04-06 15:14:21,learn_tuning.models.neural.affine_encoder,6dab24297b5f3d3adb5d44a6a2a18cf9,learn_tuning.datasets.real.mouse_static_loaders,63a570842dc1efa062d72c507a3f7348,learn_tuning.trainers.affine_trainer,281e54bbfa39beda082739f5bea78d84
2,learn_tuning.models.neural.affine_encoder,6dab24297b5f3d3adb5d44a6a2a18cf9,learn_tuning.datasets.real.mouse_static_loaders,63a570842dc1efa062d72c507a3f7348,learn_tuning.trainers.affine_trainer,1879c35492a8788d5b6eea7d4a485fdd,7,learn_tuning.models.neural.affine_encoder,6dab24297b5f3d3adb5d44a6a2a18cf9,learn_tuning.datasets.real.mouse_static_loaders,63a570842dc1efa062d72c507a3f7348,learn_tuning.trainers.affine_trainer,281e54bbfa39beda082739f5bea78d84,professional trainer.SOA Model.beautiful dataset,-0.00783618,=BLOB=,Mohammad Bashiri,2020-04-06 15:14:34,learn_tuning.models.neural.affine_encoder,6dab24297b5f3d3adb5d44a6a2a18cf9,learn_tuning.datasets.real.mouse_static_loaders,63a570842dc1efa062d72c507a3f7348,learn_tuning.trainers.affine_trainer,1879c35492a8788d5b6eea7d4a485fdd
2,learn_tuning.models.neural.affine_encoder,dc8b9e3a7843a4d30186a58af56eb81c,learn_tuning.datasets.real.mouse_static_loaders,63a570842dc1efa062d72c507a3f7348,learn_tuning.trainers.affine_trainer,1879c35492a8788d5b6eea7d4a485fdd,7,learn_tuning.models.neural.affine_encoder,6dab24297b5f3d3adb5d44a6a2a18cf9,learn_tuning.datase